In [1]:
print("⏳ Setting up environment for HMTAS inference...")

# --- Step 1: Ensure clean base ---
!pip uninstall -y bleurt tensorflow tensorflow-text -q || true
!rm -rf /root/.cache/pip /root/.cache/huggingface /root/.cache/nltk

# --- Step 2: Install core stable packages (quietly) ---
!pip install -q -U numpy==1.26.4 scikit-learn==1.3.2
!pip install -q transformers==4.41.2 datasets==2.18.0 sentence-transformers==2.7.0
!pip install -q rouge-score==0.1.2 bert-score==0.3.13
!pip install -q networkx==3.2.1 hdbscan==0.8.33 umap-learn==0.5.5
!pip install -q matplotlib seaborn nltk

# --- Step 3: Torch (CPU) install (GPU uses default runtime CUDA) ---
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

# --- Step 4: Download NLTK resources silently ---
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

print("✅ Environment ready for HMTAS inference and visualization.")


⏳ Setting up environment for HMTAS inference...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 99.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.3.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
# hmtas_unified_eval.py
# Inference-only unified evaluation with HMTAS (FLAN-T5-Base) + baselines (FLAN-T5-Base, BART, T5-Base, PEGASUS)
import warnings, os, time, gc, json, random
warnings.filterwarnings("ignore")
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

from pathlib import Path
from typing import List, Dict, Tuple
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

# Torch / HF
import torch
import torch.nn.functional as F
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Metrics
from rouge_score import rouge_scorer
from bert_score import score as bert_score_calculator
try:
    import sacrebleu
    _HAS_SACREBLEU = True
except Exception:
    _HAS_SACREBLEU = False
try:
    from nltk.translate.meteor_score import meteor_score
    _HAS_METEOR = True
except Exception:
    _HAS_METEOR = False
from nltk.tokenize import sent_tokenize

# Clustering
import hdbscan
import networkx as nx
try:
    import umap.umap_ as umap
    _HAS_UMAP = True
except Exception:
    _HAS_UMAP = False

# --------------------------
# Configuration
# --------------------------
class Config:
    total_test_samples = 100  # small, fast
    output_dir = Path("./hmtas_output_unified")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seed = 42

    # Models to evaluate
    # key: nickname; values: HF name, max_input_len, prompt style
    # prompt_style ∈ {"flan","t5","plain"}
    models = {
        "HMTAS(FLAN-T5-Base)": {"hf": "google/flan-t5-base", "max_in": 512, "prompt_style": "flan", "guided": True},
        "FLAN-T5-Base":        {"hf": "google/flan-t5-base", "max_in": 512, "prompt_style": "flan", "guided": False},
        "BART-Large-CNN":      {"hf": "facebook/bart-large-cnn", "max_in": 1024, "prompt_style": "plain", "guided": False},
        "T5-Base":             {"hf": "t5-base", "max_in": 512, "prompt_style": "t5", "guided": False},
        "PEGASUS-CNNDM":       {"hf": "google/pegasus-cnn_dailymail", "max_in": 1024, "prompt_style": "plain", "guided": False},
    }

    # Generation defaults (shared; per-model overrides possible)
    max_target_length = 160
    num_beams = 5
    length_penalty = 1.2
    min_length = 40
    repetition_penalty = 1.1
    no_repeat_ngram_size = 3

    # HMTAS guidance settings (tuned for 512-token backbones)
    token_budget_map = {
        "google/flan-t5-base": 380,
        "t5-base": 360,
        "facebook/bart-large-cnn": 420,
        "google/pegasus-cnn_dailymail": 420,
    }
    min_cluster_size = 3
    per_cluster_topk = 3
    fallback_topk = 8
    sim_threshold = 0.15
    pagerank_alpha = 0.85
    pagerank_max_iter = 100

cfg = Config()

# --------------------------
# Utilities
# --------------------------
def set_seed(seed: int):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def mkdir(p: Path): p.mkdir(parents=True, exist_ok=True)

# --------------------------
# Sentence encoder / clustering
# --------------------------
class SentenceEncoderGPU:
    def __init__(self, device: str):
        self.device = device
        self.model = SentenceTransformer("all-MiniLM-L6-v2", device=device)
        self.model.eval()
    def encode(self, sentences: List[str]) -> torch.Tensor:
        if not sentences:
            return torch.empty(0, 384, device=self.device)
        return self.model.encode(sentences, convert_to_tensor=True, device=self.device, show_progress_bar=False)
    def cosine(self, a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
        a, b = F.normalize(a, p=2, dim=-1), F.normalize(b, p=2, dim=-1)
        return torch.mm(a, b.t())

class HMTASClusterer:
    def __init__(self, cfg: Config): self.cfg = cfg
    def cluster(self, emb: torch.Tensor) -> np.ndarray:
        e = emb.detach().cpu().numpy()
        n = e.shape[0]
        if n < self.cfg.min_cluster_size: return np.zeros(n, dtype=int)
        c = hdbscan.HDBSCAN(min_cluster_size=self.cfg.min_cluster_size, min_samples=1, metric="euclidean")
        labels = c.fit_predict(e)
        if (labels == -1).any():
            non = np.where(labels != -1)[0]
            for i in np.where(labels == -1)[0]:
                sims = F.cosine_similarity(torch.tensor(e[i]).unsqueeze(0), torch.tensor(e[non]), dim=-1)
                labels[i] = int(labels[non][sims.argmax().item()])
        return labels

# --------------------------
# HMTAS Processor
# --------------------------
class HMTASProcessor:
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.encoder = SentenceEncoderGPU(cfg.device)
        self.clusterer = HMTASClusterer(cfg)
        self.reducer = umap.UMAP(n_components=16, random_state=cfg.seed) if _HAS_UMAP else None

    def _prompt(self, prompt_style: str, guide: str, docs_join: str) -> str:
        if prompt_style == "flan":
            return f"Summarize the following multiple articles into one concise paragraph:\n{guide}\n{docs_join}"
        if prompt_style == "t5":
            return f"summarize: {guide} {docs_join}"
        # plain (BART/PEGASUS)
        return f"{guide}</s><s>{docs_join}"

    def preprocess_docs(self, docs: List[str]) -> List[str]:
        sents = []
        for d in docs:
            sents.extend([s.strip() for s in sent_tokenize(d) if 8 < len(s.split()) < 120])
        return sents

    def create_guided_input(self, docs: List[str], tokenizer, hf_name: str, prompt_style: str, max_in: int) -> str:
        sents = self.preprocess_docs(docs)
        if not sents:
            text = self._prompt(prompt_style, "", " ".join(docs))
            # Hard char clamp before tokenization (saves tokens for FLAN/T5)
            text = text[:1500]
            ids = tokenizer.encode(text, add_special_tokens=False)
            if len(ids) > max_in:
                ids = ids[:max_in]; text = tokenizer.decode(ids, skip_special_tokens=True)
            return text

        # embeddings (optional UMAP)
        emb = self.encoder.encode(sents)
        emb_np = emb.detach().cpu().numpy()
        if self.reducer is not None:
            try:
                emb_np = self.reducer.fit_transform(emb_np)
                emb = torch.tensor(emb_np, device=self.cfg.device)
            except Exception:
                pass

        sim = self.encoder.cosine(emb, emb)
        labels = self.clusterer.cluster(emb)

        chosen = []
        for l in np.unique(labels):
            idxs = np.where(labels == l)[0].tolist()
            sub = sim[idxs][:, idxs].detach().cpu().numpy()
            sub[sub < self.cfg.sim_threshold] = 0
            np.fill_diagonal(sub, 0)
            try:
                G = nx.from_numpy_array(sub)
                pr = nx.pagerank(G, alpha=self.cfg.pagerank_alpha, max_iter=self.cfg.pagerank_max_iter)
                ranked = sorted(pr.items(), key=lambda x: x[1], reverse=True)
                chosen.extend([idxs[i] for i, _ in ranked[:self.cfg.per_cluster_topk]])
            except Exception:
                sums = sub.sum(axis=1)
                top = np.argsort(-sums)[:self.cfg.per_cluster_topk]
                chosen.extend([idxs[int(i)] for i in top])

        chosen = list(dict.fromkeys(chosen))
        if len(chosen) < self.cfg.fallback_topk:
            sums = sim.sum(dim=1).detach().cpu().numpy()
            topk = np.argsort(-sums)[:self.cfg.fallback_topk]
            chosen = list(dict.fromkeys(chosen + topk.tolist()))

        token_budget = self.cfg.token_budget_map.get(hf_name, 360)
        total, final = 0, []
        for i in chosen:
            n = len(tokenizer.encode(sents[i], add_special_tokens=False))
            if total + n > token_budget: break
            final.append(i); total += n

        guide = " ".join([sents[i] for i in final])
        docs_join = " ".join(f"[DOC_{i}] {d}" for i, d in enumerate(docs)) if prompt_style != "plain" else " ".join(docs)
        text = self._prompt(prompt_style, guide, docs_join)

        # extra char clamp before tokenization (esp. for 512-token models)
        if max_in <= 512:
            text = text[:1500]

        ids = tokenizer.encode(text, add_special_tokens=False)
        if len(ids) > max_in:
            ids = ids[:max_in]
            text = tokenizer.decode(ids, skip_special_tokens=True)
        return text

# --------------------------
# Generation helpers
# --------------------------
def batch_generate(model, tok, texts, max_input_len, cfg: Config):
    outs = []
    bs = 4
    for i in range(0, len(texts), bs):
        chunk = texts[i:i+bs]
        enc = tok(chunk, max_length=max_input_len, truncation=True, padding=True, return_tensors="pt").to(cfg.device)
        with torch.no_grad():
            out_ids = model.generate(
                **enc,
                num_beams=cfg.num_beams,
                max_length=cfg.max_target_length,
                min_length=cfg.min_length,
                length_penalty=cfg.length_penalty,
                repetition_penalty=cfg.repetition_penalty,
                no_repeat_ngram_size=cfg.no_repeat_ngram_size,
            )
        outs.extend(tok.batch_decode(out_ids, skip_special_tokens=True))
    return [o.strip() for o in outs]

# --------------------------
# Metrics
# --------------------------
def compute_all_metrics(refs: List[str], cands: List[str], sources: List[str], tok) -> Dict[str, List[float]]:
    rouge = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
    r1, r2, rL, bleus, meteors, covs, comps = [], [], [], [], [], [], []
    for src, ref, can in zip(sources, refs, cands):
        sc = rouge.score(ref, can)
        r1.append(sc['rouge1'].fmeasure)
        r2.append(sc['rouge2'].fmeasure)
        rL.append(sc['rougeL'].fmeasure)
        # BLEU
        if _HAS_SACREBLEU:
            try: bleus.append(sacrebleu.sentence_bleu(can, [ref]).score)
            except: bleus.append(0.0)
        else:
            bleus.append(0.0)
        # METEOR
        if _HAS_METEOR:
            try: meteors.append(meteor_score([ref], can) * 100.0)
            except: meteors.append(0.0)
        else:
            meteors.append(0.0)
        # Coverage & Compression
        src_t = len(tok.encode(src, add_special_tokens=False))
        sum_t = len(tok.encode(can, add_special_tokens=False))
        coverage = (sum_t / src_t) * 100.0 if src_t > 0 else 0.0
        compression = (src_t / max(sum_t, 1)) if sum_t > 0 else 0.0
        covs.append(coverage); comps.append(compression)

    # BERTScore
    try:
        _, _, F1 = bert_score_calculator(cands, refs, lang='en', model_type='distilbert-base-uncased', device=cfg.device, verbose=False)
        bsf1 = (F1.detach().cpu().numpy() * 100.0).tolist()
    except Exception:
        bsf1 = [0.0]*len(cands)

    return {
        "rouge1": r1, "rouge2": r2, "rougeL": rL,
        "bertscore_f1": bsf1,
        "bleu": bleus, "meteor": meteors,
        "coverage": covs, "compression": comps,
    }

# --------------------------
# Data: build multi-document pool
# --------------------------
def load_multi_document_dataset(total: int):
    print("Loading multi-document subset...")
    ds1 = load_dataset("cnn_dailymail", "3.0.0", split="test[:1200]")
    ds2 = load_dataset("multi_news", split="test[:1200]")
    ds3 = load_dataset("xsum", split="test[:1200]")
    pool = []
    for i in range(total):
        docs = [ds1[i]['article'], ds2[i]['document'], ds3[i]['document']]
        # use MultiNews summary as reference (multi-doc)
        pool.append({"texts": docs, "summary": ds2[i]['summary']})
    print(f"✓ Built {len(pool)} multi-document examples")
    return pool

# --------------------------
# Plotting
# --------------------------
def plot_metric_bars(results_summary: Dict[str, Dict], save_path: Path):
    metrics = ['rouge1_mean','rouge2_mean','rougeL_mean','bertscore_mean','bleu_mean','meteor_mean']
    df = pd.DataFrame(results_summary).T
    df_plot = df[[m for m in df.columns if m in metrics]]
    df_plot.columns = ['ROUGE-1','ROUGE-2','ROUGE-L','BERTScore','BLEU','METEOR']
    ax = df_plot.plot(kind='bar', figsize=(12,6), rot=0, edgecolor='black', alpha=0.9)
    ax.set_ylabel('Score (%)')
    plt.title('Model Metric Comparison')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150); plt.close()

def plot_rouge_violin(all_scores: Dict[str, Dict], save_path: Path):
    df = pd.DataFrame({k: np.array(v['rougeL']) * 100 for k, v in all_scores.items()})
    plt.figure(figsize=(10,5)); sns.violinplot(data=df, inner='quartile')
    plt.title('ROUGE-L Distribution Across Models'); plt.ylabel('ROUGE-L (%)')
    plt.tight_layout(); plt.savefig(save_path, dpi=150); plt.close()

def plot_correlation_heatmap(all_scores: Dict[str, Dict], save_path: Path):
    rows = []
    for m, v in all_scores.items():
        rows.append({
            'model': m,
            'rouge1': np.mean(v['rouge1']) * 100,
            'rouge2': np.mean(v['rouge2']) * 100,
            'rougeL': np.mean(v['rougeL']) * 100,
            'bert': np.mean(v['bertscore_f1']),
            'bleu': np.mean(v['bleu']),
            'meteor': np.mean(v['meteor'])
        })
    df = pd.DataFrame(rows).set_index('model')
    corr = df.corr()
    plt.figure(figsize=(8,6)); sns.heatmap(corr, annot=True, fmt='.2f', cmap='vlag', center=0)
    plt.title('Correlation Between Metrics (model-mean)')
    plt.tight_layout(); plt.savefig(save_path, dpi=150); plt.close()

def plot_radar(results_summary: Dict[str, Dict], save_path: Path):
    labels = ['ROUGE-1','ROUGE-2','ROUGE-L','BERTScore','BLEU','METEOR']
    n = len(labels)
    angles = np.linspace(0, 2*np.pi, n, endpoint=False).tolist()
    angles += angles[:1]
    plt.figure(figsize=(7,7)); ax = plt.subplot(polar=True)
    for model_name, vals in results_summary.items():
        stats = [vals['rouge1_mean'], vals['rouge2_mean'], vals['rougeL_mean'], vals['bertscore_mean'], vals['bleu_mean'], vals['meteor_mean']]
        stats += stats[:1]
        ax.plot(angles, stats, label=model_name); ax.fill(angles, stats, alpha=0.1)
    ax.set_thetagrids(np.degrees(angles[:-1]), labels)
    ax.set_title('Radar: Model Comparison'); ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    plt.tight_layout(); plt.savefig(save_path, dpi=150); plt.close()

# --------------------------
# Main
# --------------------------
def main():
    set_seed(cfg.seed)
    mkdir(cfg.output_dir)

    print("="*90)
    print("🚀 HMTAS — Unified Inference & Comparison (FLAN-T5-Base guided vs baselines)")
    print("="*90)
    print(f"Device: {cfg.device}")
    print(f"Samples: {cfg.total_test_samples}")

    # Data
    pool = load_multi_document_dataset(cfg.total_test_samples)
    refs = [p['summary'] for p in pool]
    sources = [" ".join(p['texts']) for p in pool]

    # HMTAS processor
    hmtas_proc = HMTASProcessor(cfg)

    all_results = {}
    rows_for_csv = []

    # Evaluate each model
    for nick, spec in cfg.models.items():
        hf_name = spec["hf"]; max_in = spec["max_in"]; style = spec["prompt_style"]; guided = spec["guided"]
        print(f"\n🔧 Loading: {nick}  ({hf_name})")
        tok = AutoTokenizer.from_pretrained(hf_name)
        mdl = AutoModelForSeq2SeqLM.from_pretrained(hf_name).to(cfg.device)
        mdl.eval()

        # Build inputs
        print(f"📊 Evaluating {nick} ...")
        t0 = time.time()
        if guided:
            inputs = [hmtas_proc.create_guided_input(p['texts'], tok, hf_name, style, max_in) for p in pool]
        else:
            # base prompt per style (flat multi-doc)
            inputs = []
            for p in pool:
                docs = " ".join(p['texts'])
                if style == "flan":
                    txt = f"Summarize the following multiple articles into one concise paragraph:\n{docs}"
                elif style == "t5":
                    txt = f"summarize: {docs}"
                else:
                    txt = docs  # plain
                # safeguard for short-context models
                if max_in <= 512:
                    txt = txt[:1500]
                ids = tok.encode(txt, add_special_tokens=False)
                if len(ids) > max_in:
                    ids = ids[:max_in]; txt = tok.decode(ids, skip_special_tokens=True)
                inputs.append(txt)

        cands = batch_generate(mdl, tok, inputs, max_in, cfg)
        elapsed = time.time() - t0
        metrics = compute_all_metrics(refs, cands, sources, tok)
        all_results[nick] = metrics

        # store outputs
        for i, (cand, ref) in enumerate(zip(cands, refs)):
            rows_for_csv.append({"model": nick, "idx": i, "candidate": cand, "reference": ref})

        # Free VRAM
        del mdl; gc.collect()
        if torch.cuda.is_available(): torch.cuda.empty_cache()

        print(f"→ {nick} done. time: {elapsed:.1f}s")

    # Summaries
    results_summary = {}
    for name, res in all_results.items():
        results_summary[name] = {
            'rouge1_mean': float(np.mean(res['rouge1']) * 100), 'rouge1_std': float(np.std(res['rouge1']) * 100),
            'rouge2_mean': float(np.mean(res['rouge2']) * 100), 'rouge2_std': float(np.std(res['rouge2']) * 100),
            'rougeL_mean': float(np.mean(res['rougeL']) * 100), 'rougeL_std': float(np.std(res['rougeL']) * 100),
            'bertscore_mean': float(np.mean(res['bertscore_f1'])), 'bertscore_std': float(np.std(res['bertscore_f1'])),
            'bleu_mean': float(np.mean(res['bleu'])), 'bleu_std': float(np.std(res['bleu'])),
            'meteor_mean': float(np.mean(res['meteor'])), 'meteor_std': float(np.std(res['meteor'])),
            'coverage_mean': float(np.mean(res['coverage'])), 'compression_mean': float(np.mean(res['compression'])),
        }

    # Save CSV of generated summaries
    df_out = pd.DataFrame(rows_for_csv)
    df_out.to_csv(cfg.output_dir / "generated_summaries.csv", index=False)

    # Save JSONs
    with open(cfg.output_dir / "results_summary.json", "w") as f:
        json.dump(results_summary, f, indent=2)
    # Sample raw (truncate arrays for size)
    sample_raw = {}
    for k, v in all_results.items():
        sample_raw[k] = {kk: (vv[:100] if isinstance(vv, list) else vv) for kk, vv in v.items()}
    with open(cfg.output_dir / "results_raw_sample.json", "w") as f:
        json.dump(sample_raw, f, indent=2)

    # Print table
    print("\n" + "="*90)
    print("📊 FINAL RESULTS SUMMARY (means)")
    print("="*90)
    print("| Model | R1 | R2 | RL | BERT | BLEU | METEOR | Cover | Compr |")
    print("|-------|----|----|----|------|------|--------|-------|-------|")
    for name, v in results_summary.items():
        print(f"| {name:<20} | {v['rouge1_mean']:.2f} | {v['rouge2_mean']:.2f} | {v['rougeL_mean']:.2f} | "
              f"{v['bertscore_mean']:.2f} | {v['bleu_mean']:.2f} | {v['meteor_mean']:.2f} | "
              f"{v['coverage_mean']:.2f} | {v['compression_mean']:.2f} |")

    # Plots
    plot_dir = cfg.output_dir / "plots"; mkdir(plot_dir)
    try: plot_metric_bars(results_summary, plot_dir / "metric_comparison.png")
    except Exception as e: print("plot_metric_bars failed:", e)
    try: plot_rouge_violin(all_results, plot_dir / "rouge_violin.png")
    except Exception as e: print("plot_rouge_violin failed:", e)
    try: plot_correlation_heatmap(all_results, plot_dir / "metric_correlation.png")
    except Exception as e: print("plot_correlation_heatmap failed:", e)
    try: plot_radar(results_summary, plot_dir / "radar_comparison.png")
    except Exception as e: print("plot_radar failed:", e)

    print(f"\n✓ Saved outputs to {cfg.output_dir}")

if __name__ == "__main__":
    main()


🚀 HMTAS — Unified Inference & Comparison (FLAN-T5-Base guided vs baselines)
Device: cuda
Samples: 100
Loading multi-document subset...


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

...

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3436 > 1024). Running this sequence through the model will result in indexing errors


📊 Evaluating PEGASUS-CNNDM ...
→ PEGASUS-CNNDM done. time: 65.0s

📊 FINAL RESULTS SUMMARY (means)
| Model | R1 | R2 | RL | BERT | BLEU | METEOR | Cover | Compr |
|-------|----|----|----|------|------|--------|-------|-------|
| HMTAS(FLAN-T5-Base)  | 15.33 | 3.05 | 9.22 | 70.43 | 0.00 | 0.00 | 2.44 | 71.57 |
| FLAN-T5-Base         | 12.10 | 2.02 | 7.86 | 68.11 | 0.00 | 0.00 | 2.36 | 65.17 |
| BART-Large-CNN       | 11.90 | 1.40 | 7.75 | 68.92 | 0.00 | 0.00 | 2.14 | 64.91 |
| T5-Base              | 10.83 | 1.15 | 7.40 | 68.20 | 0.00 | 0.00 | 2.17 | 68.18 |
| PEGASUS-CNNDM        | 11.04 | 1.43 | 7.52 | 67.95 | 0.00 | 0.00 | 2.30 | 64.00 |

✓ Saved outputs to hmtas_output_unified
